<a href="https://colab.research.google.com/github/dgsmith1988/ECSE-552-Final-Project/blob/rubert/execution_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overall Script Description
Run through each of the different cells to get things working. What should happen is that you connect to Google Drive, some data is transferred and then unzipped and then the model is trained on it and outputs data indicating correct operation. On the Google Drive there should also be files added under "Measurements" which have a date and time-stamp corresponding to the output of training process. The time stamps are generated by the Google Colab instance so they won't match exactly with your local time depending on where the instance they supplied us is located.

The cells are broken up based on functionality to make it easier to debug/profile/troubleshoot any issues which might come up.

# Configure Directories/Paths/Languages
The following cell is where you configure which languages you want to work with (from the three which are available on Google Drive at the moment). It is also where you set the paths to where the Google Drive will be mounted and where the data is extracted to.


In [1]:
import os
languages = ["EN", "DE", "ES"]
mount_point = '/content/drive'
load_path = os.path.join(mount_point, 'MyDrive/ECSE-552-FP/Data/')
unzip_path = '/content/speech_data'
save_path = '/MyDrive/ECSE-552-FP/Measurements'

from google.colab import drive
drive.mount(mount_point)

Mounted at /content/drive


# Download and extract the dataset zip files
The following downloads and extracts the language zip files from Google Drive. It is done separately in it's own cell to make it easier to profile/debug things if there are issues in the network connection between Google Drive and Google Colab. 

This is separate from dataset creation for two reasons. The first is that if the network accesses are obscured by accessing the data via the network drive, it is much harder to debug bottlenecks in creating the dataset. The second is that all of the data must be transferred once at somepoint during one epoch, so it's easier to do it all up front and make sure the data is on the Colab instance before training. Localizing as much as possible to the Colab instance during training makes things easier to debug as well as more efficient.

There is a commented out code which allows you to select between the "debug" dataset and the "full" dataset for the languages. The "debug" set consists of a smaller subset (20 samples for each language) to aid in debugging the dataflow in models. It alleviates the need to run through the entire dataset and helps ensure all the pipes are connected correctly.

In [2]:
for language in languages:
    language_dir_path = os.path.join(unzip_path, language)
    os.makedirs(language_dir_path, exist_ok=True)
    archive = language + "_debug_set" + ".zip"
    #archive = language + ".zip"
    !unzip {os.path.join(load_path, archive)} -d {language_dir_path}

Archive:  /content/drive/MyDrive/ECSE-552-FP/Data/EN.zip
   creating: /content/speech_data/EN/1028-20100710-hne/
   creating: /content/speech_data/EN/1028-20100710-hne/wav/
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/ar-01.wav  
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/ar-02.wav  
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/ar-03.wav  
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/ar-04.wav  
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/ar-05.wav  
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/ar-06.wav  
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/ar-07.wav  
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/rp-29.wav  
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/rp-30.wav  
  inflating: /content/speech_data/EN/1028-20100710-hne/wav/rp-31.wav  
   creating: /content/speech_data/EN/1337ad-20170321-ajg/
   creating: /content/speech_data/EN/1337ad-20170321-ajg/wa

# Download the source code from GitHub

In [3]:
user = "dgsmith1988"
repo = "ECSE-552-Final-Project"
src_dir = "Code"
pyfiles = ["models.py", "train.py", "dict_logger.py", "feature_extraction.py"]

for pyfile in pyfiles:
    url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{src_dir}/{pyfile}"
    !wget {url}


--2022-03-18 12:38:14--  https://raw.githubusercontent.com/dgsmith1988/ECSE-552-Final-Project/main/Code/models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5608 (5.5K) [text/plain]
Saving to: ‘models.py’

models.py           100%[===================>]   5.48K  --.-KB/s    in 0s      

2022-03-18 12:38:14 (33.2 MB/s) - ‘models.py’ saved [5608/5608]

--2022-03-18 12:38:14--  https://raw.githubusercontent.com/dgsmith1988/ECSE-552-Final-Project/main/Code/train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7679 (7.5K) [text/

# Install the supporting/required libraries



In [ ]:
!pip install torchmetrics
!pip install pytorch-lightning
!pip install librosa

     |████████████████████████████████| 397 kB 4.1 MB/s 


# Run/Train the Model

In [ ]:
import train
from models import BaselineResnetClassifier

model = BaselineResnetClassifier(num_classes=3)
train.train_voxforge_classifier(model, data_dir=unzip_path)